## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config1 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.7826,147.0587,70.0,33,30,8.99,scattered clouds
1,1,Mahebourg,MU,-20.4081,57.7000,77.0,88,40,5.75,scattered clouds
2,2,Saldanha,ZA,-33.0117,17.9442,73.4,64,0,3.44,clear sky
3,3,Hermanus,ZA,-34.4187,19.2345,70.0,85,47,8.99,scattered clouds
4,4,Avarua,CK,-21.2078,-159.7750,84.2,74,75,20.71,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [27]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.7826,147.0587,70.00,33,30,8.99,scattered clouds
1,1,Mahebourg,MU,-20.4081,57.7000,77.00,88,40,5.75,scattered clouds
2,2,Saldanha,ZA,-33.0117,17.9442,73.40,64,0,3.44,clear sky
3,3,Hermanus,ZA,-34.4187,19.2345,70.00,85,47,8.99,scattered clouds
4,4,Avarua,CK,-21.2078,-159.7750,84.20,74,75,20.71,broken clouds
5,5,Samarai,PG,-10.6167,150.6667,84.94,74,100,7.14,moderate rain
8,8,Georgetown,MY,5.4112,100.3354,77.00,88,40,3.44,light rain
11,11,Rikitea,PF,-23.1203,-134.9692,78.08,75,94,11.92,overcast clouds
12,12,Bengkulu,ID,-3.8004,102.2655,77.94,77,100,4.25,light rain
16,16,Faanui,PF,-16.4833,-151.7500,81.61,76,100,16.53,light rain


In [28]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                244
City                   244
Country                243
Lat                    244
Lng                    244
Max Temp               244
Humidity               244
Cloudiness             244
Wind Speed             244
Current Description    244
dtype: int64

In [29]:
#4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [23]:
preferred_cities_df.count()

City_ID                243
City                   243
Country                243
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Current Description    243
dtype: int64

In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()


# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)



,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,New Norfolk,AU,70.00,scattered clouds,-42.7826,147.0587,
1,Mahebourg,MU,77.00,scattered clouds,-20.4081,57.7000,
2,Saldanha,ZA,73.40,clear sky,-33.0117,17.9442,
3,Hermanus,ZA,70.00,scattered clouds,-34.4187,19.2345,
4,Avarua,CK,84.20,broken clouds,-21.2078,-159.7750,
5,Samarai,PG,84.94,moderate rain,-10.6167,150.6667,
8,Georgetown,MY,77.00,light rain,5.4112,100.3354,
11,Rikitea,PF,78.08,overcast clouds,-23.1203,-134.9692,
12,Bengkulu,ID,77.94,light rain,-3.8004,102.2655,
16,Faanui,PF,81.61,light rain,-16.4833,-151.7500,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    

    # 6e. Make request and retrieve the JSON data from the search.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
   
    
# 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")   
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
#7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"]!=""]

hotel_df["Hotel Name"].replace("", np.nan, inplace=True)
hotel_df.dropna(subset=["Hotel Name"], inplace=True)

clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,New Norfolk,AU,70.0,scattered clouds,-42.7826,147.0587,The Shingles Riverside Cottages
1,Mahebourg,MU,77.0,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
2,Saldanha,ZA,73.4,clear sky,-33.0117,17.9442,Blue Bay Lodge
3,Hermanus,ZA,70.0,scattered clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
4,Avarua,CK,84.2,broken clouds,-21.2078,-159.7750,Paradise Inn


In [40]:
# Checking no empty cells
clean_hotel_df.count()

City                   230
Country                230
Max Temp               230
Current Description    230
Lat                    230
Lng                    230
Hotel Name             230
dtype: int64

In [41]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/patriciacb1/Desktop/UC_Berkeley_Data_Analysis_Bootcamp/World_Weather_Analysis/Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [43]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))